In [1]:
!pip install numpy opencv-python scipy pygame

In [2]:
import numpy as np
import cv2
import scipy
import pygame
import matplotlib.pyplot as plt
import math
import random


pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [19]:


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display, Video

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0

def a_star(maze, start, end):
    start_node = Node(start)
    end_node = Node(end)
    open_list = []
    closed_list = []
    open_list.append(start_node)

    while len(open_list) > 0:
        current_node = open_list[0]
        current_index = 0
        for index, node in enumerate(open_list):
            if node.f < current_node.f:
                current_node = node
                current_index = index

        open_list.pop(current_index)
        closed_list.append(current_node)

        if current_node.position == end_node.position:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1]

        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[0]) - 1) or node_position[1] < 0:
                continue
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            new_node = Node(node_position, current_node)
            children.append(new_node)

        for child in children:
            if child in closed_list:
                continue
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            if any(open_node for open_node in open_list if child.position == open_node.position and child.g > open_node.g):
                continue

            open_list.append(child)

class EKF_SLAM:
    def __init__(self, initial_state=[0, 0, 0]):
        self.mu = np.array(initial_state).reshape((3, 1))
        self.sigma = np.eye(3) * 0.1

    def predict(self, control, dt=1.0):
        v, omega = control
        theta = self.mu[2, 0]

        self.mu[0, 0] += v * np.cos(theta) * dt
        self.mu[1, 0] += v * np.sin(theta) * dt
        self.mu[2, 0] += omega * dt

        G = np.array([[1, 0, -v * np.sin(theta) * dt],
                      [0, 1, v * np.cos(theta) * dt],
                      [0, 0, 1]])

        R = np.eye(3) * 0.1
        self.sigma = G @ self.sigma @ G.T + R

    def update(self, z, landmark_pos):
        q = (landmark_pos[0] - self.mu[0, 0]) ** 2 + (landmark_pos[1] - self.mu[1, 0]) ** 2
        z_hat = np.array([[np.sqrt(q)],
                          [np.arctan2(landmark_pos[1] - self.mu[1, 0], landmark_pos[0] - self.mu[0, 0]) - self.mu[2, 0]]])

        H = np.array([[-(landmark_pos[0] - self.mu[0, 0]) / np.sqrt(q), -(landmark_pos[1] - self.mu[1, 0]) / np.sqrt(q), 0],
                      [(landmark_pos[1] - self.mu[1, 0]) / q, -(landmark_pos[0] - self.mu[0, 0]) / q, -1]])

        Q = np.eye(2) * 0.1
        S = H @ self.sigma @ H.T + Q
        K = self.sigma @ H.T @ np.linalg.inv(S)

        self.mu += K @ (z - z_hat)
        self.sigma = (np.eye(len(self.mu)) - K @ H) @ self.sigma

    def get_pose(self):
        return self.mu.flatten()

def visualize_slam_path(map_grid, path, start, goal, ekf_slam, filename='slam_path_animation.mp4'):
    fig, ax = plt.subplots(figsize=(8, 8))
    for i in range(map_grid.shape[0]):
        for j in range(map_grid.shape[1]):
            if map_grid[i, j] == 1:
                ax.add_patch(patches.Rectangle((j, i), 1, 1, fill=True, color='black'))
            else:
                ax.add_patch(patches.Rectangle((j, i), 1, 1, fill=True, color='white'))

    ax.plot(start[1] + 0.5, start[0] + 0.5, "go", markersize=15, label="Start")
    ax.plot(goal[1] + 0.5, goal[0] + 0.5, "ro", markersize=15, label="Goal")

    plt.xlim(0, map_grid.shape[1])
    plt.ylim(0, map_grid.shape[0])
    plt.gca().set_aspect('equal')
    plt.gca().invert_yaxis()
    plt.legend()
    plt.grid()

    line, = ax.plot([], [], "b-", linewidth=2, label="Path")
    robot, = ax.plot([], [], "bo", markersize=10)

    def animate(i):
        if i < len(path):
            p = path[:i+1]
            ekf_slam.predict(control=[0.5, 0.05])
            line.set_data([pt[1] + 0.5 for pt in p], [pt[0] + 0.5 for pt in p])
            robot_pose = ekf_slam.get_pose()
            robot.set_data(robot_pose[0] + 0.5, robot_pose[1] + 0.5)
        return line, robot

    anim = FuncAnimation(fig, animate, frames=len(path), interval=500, blit=True)

    anim.save(filename, writer='ffmpeg', fps=2)
    plt.close(fig)
    return HTML(anim.to_jshtml()), filename

map_grid = np.array([
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 1, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
])

start_pos = (4, 3)
goal_pos = (9, 9)

path_result = a_star(map_grid, start_pos, goal_pos)
print("Path found:", path_result)

ekf_slam = EKF_SLAM(initial_state=[0, 0, 0])

animation_html, saved_filename = visualize_slam_path(map_grid, path_result, start_pos, goal_pos, ekf_slam)
display(animation_html)

Video(saved_filename)


Path found: [(4, 3), (4, 2), (5, 2), (6, 2), (6, 3), (7, 3), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (9, 7), (9, 8), (9, 9)]


<ipython-input-19-9d889e5e5a47>:132: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  robot.set_data(robot_pose[0] + 0.5, robot_pose[1] + 0.5)
<ipython-input-19-9d889e5e5a47>:132: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  robot.set_data(robot_pose[0] + 0.5, robot_pose[1] + 0.5)
<ipython-input-19-9d889e5e5a47>:132: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  robot.set_data(robot_pose[0] + 0.5, robot_pose[1] + 0.5)


In [18]:
map_grid = np.array([
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 1, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
])

start_pos = (0, 0)  # Start position
goal_pos = (9, 9)   # Goal position

path_result = a_star(map_grid, start_pos, goal_pos)
print("Path found:", path_result)

# Initialize EKF SLAM
ekf_slam = EKF_SLAM(initial_state=[0, 0, 0])

# Visualize and save the SLAM path animation, then display it
animation_html, saved_filename = visualize_slam_path(map_grid, path_result, start_pos, goal_pos, ekf_slam)
display(animation_html)

# Display the saved video file for download
Video(saved_filename)

Path found: [(4, 3), (4, 2), (5, 2), (6, 2), (6, 3), (7, 3), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (9, 7), (9, 8), (9, 9)]


<ipython-input-16-57e75f3e25cb>:148: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  robot.set_data(robot_pose[0] + 0.5, robot_pose[1] + 0.5)  # Update robot position based on EKF estimate
<ipython-input-16-57e75f3e25cb>:148: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  robot.set_data(robot_pose[0] + 0.5, robot_pose[1] + 0.5)  # Update robot position based on EKF estimate
<ipython-input-16-57e75f3e25cb>:148: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  robot.set_data(robot_pose[0] + 0.5, robot_pose[1] + 0.5)  # Update robot position based on EKF estimate
